In [1]:
import requests
from datetime import datetime
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import plotly.express as px

In [2]:
base_url = os.getenv("IMMICH_BASE_URL")
token = os.getenv("IMMICH_TOKEN")
year = os.getenv("YEAR")
start_date = f"{year}-01-01"
end_date = f"{year}-12-31"

headers = {"x-api-key": token}
response = requests.get(f"{base_url}/asset", headers=headers)

In [ ]:
if response.status_code == 200:
    photos = response.json()
    print(len(photos))
    print(photos[200])
    # print(f"ID: {photos[0]['id']}, File Name: {photos[0]['fileName']}, Date: {photos[0]['uploadDate']}")
else:
    print(f"Error: {response.status_code} - {response.text}")


In [4]:
photos_with_metadata = [photo for photo in photos if photo['exifInfo']['latitude'] != None]

In [ ]:
print(f"{len(photos_with_metadata)}/{len(photos)}")

In [6]:
df = pd.DataFrame(photos_with_metadata)

In [7]:
exif_cols = pd.json_normalize(df.exifInfo)
df[exif_cols.columns] = exif_cols

In [ ]:
df.columns

In [9]:
filtered_df = df[(df['fileCreatedAt'] >= f"{year}-01-01") & (df['fileCreatedAt'] <= f"{year}-12-31")]

In [10]:
filtered_df = filtered_df[['fileCreatedAt', 'latitude', 'longitude', 'city', 'state', 'country']].iloc[::-1]

In [11]:
filtered_df['prev_latitude'] = [np.nan] + list(filtered_df['latitude'])[:-1]
filtered_df['prev_longitude'] = [np.nan] + list(filtered_df['longitude'])[:-1]

In [ ]:
filtered_df

In [13]:
filtered_df['distance_from_prev'] = filtered_df.apply(lambda point: geodesic((point['latitude'], point['longitude']), (point['prev_latitude'], point['prev_longitude'])).km if np.isfinite(point['prev_latitude']) else 0, axis=1)

In [14]:
final_df = filtered_df[filtered_df['distance_from_prev'] > 70]

In [ ]:
final_df

In [16]:
unique_cities = final_df.groupby(['city', 'state']).ngroups
unique_states = final_df['state'].nunique()
unique_countries = final_df['country'].nunique()
total_distance_traveled = round(final_df['distance_from_prev'].sum(), 2)
most_common_state = final_df['state'].mode().item()

In [ ]:
print(f"{year} Stats")
print(f"{unique_cities} unique cities")
print(f"{unique_states} unique states")
print(f"{unique_countries} unique countries")
print(f"{total_distance_traveled}km total distance traveled")
print(f"{most_common_state} is the most visited state")

In [105]:
i = 0
group = 0
animated_json = {'latitude': [], 'longitude': [], 'fileCreatedAt': [], 'city': [], 'state': [], 'country': [], 'group': []}
for index, row in final_df.iterrows():
    animated_json['latitude'].append(row['latitude'])
    animated_json['longitude'].append(row['longitude'])
    animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
    animated_json['city'].append(row['city'])
    animated_json['state'].append(row['state'])
    animated_json['country'].append(row['country'])
    animated_json['group'].append(group)

    if i > 0 and i < len(final_df) - 1:
        group+=1
        animated_json['latitude'].append(row['latitude'])
        animated_json['longitude'].append(row['longitude'])
        animated_json['fileCreatedAt'].append(row['fileCreatedAt'])
        animated_json['city'].append(row['city'])
        animated_json['state'].append(row['state'])
        animated_json['country'].append(row['country'])
        animated_json['group'].append(group)

    i+=1
    


In [ ]:
animated_df = pd.DataFrame(animated_json)

In [122]:
fig = px.line_map(animated_df,
            lat='latitude',
            lon='longitude',
            animation_frame='group',
            zoom=4,
            width=1200,
            height=800)

In [ ]:
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()